# Micro-C Contact vs Distance plot

Reference (https://cooltools.readthedocs.io/en/latest/notebooks/contacts_vs_distance.html)

In [1]:
# INSTALL REQUIRED PACKAGES

#!pip install cooltools==0.7.0
#!pip install seaborn
#!pip install cooler

In [1]:
# Standard library imports
import os
import warnings
from itertools import combinations
from multiprocessing import Pool
import subprocess

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import bioframe
import cooler
import cooltools
from packaging import version

# Jupyter magic (only in notebooks)
%matplotlib inline

# ------------------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------------------

# Suppress non-critical warnings
warnings.filterwarnings('ignore')

# Determine number of CPUs
num_cpus = os.cpu_count() or 1

print(f'Using {num_cpus} CPU core{'s' if num_cpus > 1 else ''}.')

Using 10 CPU cores.


## Parameters

In [2]:
REFDIR = 'reference'
WORKDIR = 'MicroC_CvD'

# Use bioframe to fetch the genomic features from the UCSC.
mm10_chromsizes = bioframe.fetch_chromsizes('mm10')
mm10_cens = pd.read_csv(f'{REFDIR}/mm10/mm10.centromere.txt', sep = '\t')

# create a view with chromosome arms using chromosome sizes and definition of centromeres
mm10_arms = bioframe.make_chromarms(mm10_chromsizes,  mm10_cens)


## Calculating CvD with downsampled matrix
All samples will be downsampled to same readcount and KR normalized for proper comparison.

In [3]:
def process_cvd(MCOOLDIR, COOLDIR, OUTDIR, NAME, RES, DSCOUNT, mm10_arms, num_cpus, WORKDIR):
    '''
    1. Subsamples the merged .mcool at resolution RES to DSCOUNT reads (if needed)
    2. KR-normalizes the subsampled matrix via normCool.sh (if needed)
    3. Computes the expected cis-decay curve, smooths it, masks dist<2, 
       merges duplicates, and writes out both the merged curve and its log-log derivative.

    Parameters
    ----------
    MCOOLDIR : str
        Path to the pooled .mcool file (expects {NAME}_allRes.mcool).
    COOLDIR : str
        Directory for normalized/downsamped .cool files.
    OUTDIR : str
        Directory for output TSVs.
    NAME : str
        Sample name prefix.
    RES : int
        Resolution in bp.
    DSCOUNT : int
        Number of contacts to subsample (e.g. 5_000_000).
    mm10_arms : pd.DataFrame
        A view_df defining chromosome arms.
    num_cpus : int
        Number of processes to use for cooltools.sample and expected_cis.
    WORKDIR : str
        Directory containing `normCool.sh`.
    '''
    os.makedirs(OUTDIR, exist_ok=True)

    # paths
    subsampled = f'{COOLDIR}/{NAME}_{RES}bp_downsampled-{DSCOUNT//1_000_000}M.cool'
    normalized = f'{COOLDIR}/{NAME}_{RES}bp_downsampled-{DSCOUNT//1_000_000}M_KR.cool'

    # calculate subsampling fraction
    pre_norm = cooler.Cooler(f'{COOLDIR}/{NAME}_{RES}bp_KR.cool')
    frac = DSCOUNT / pre_norm.info['sum']
    print(f'prenorm: {pre_norm.info['sum']}')
    print(f'sampling fraction: {frac}')
    
    # 1) subsample if needed
    if not os.path.exists(subsampled):
        print(f'sampling to {subsampled}')
        mc = cooler.Cooler(f'{MCOOLDIR}/{NAME}_allRes.mcool::/resolutions/{RES}')
        cooltools.sample(mc,
                         out_clr_path=subsampled,
                         frac=frac,
                         nproc=num_cpus)
    else:
        print(f'skipping sample (exists): {subsampled}')

    # 2) KR-normalize if needed
    if not os.path.exists(normalized):
        script = os.path.join(WORKDIR, 'normCool.sh')
        print(f'normalizing to {normalized}')
        subprocess.run([script, subsampled, normalized], check=True)
    else:
        print(f'skipping normalize (exists): {normalized}')

    # 3) compute expected cis-decay curve
    clr = cooler.Cooler(normalized)
    cvd = cooltools.expected_cis(
        clr=clr,
        view_df=mm10_arms,
        smooth=True,
        aggregate_smoothed=True,
        smooth_sigma=0.1,
        nproc=num_cpus
    )

    # write smoothed curve
    out_smooth = os.path.join(
        OUTDIR,
        f'{NAME}_{RES}bp_downsampled-{DSCOUNT//1_000_000}M_cvd_smooth.tsv'
    )
    cvd.to_csv(out_smooth, sep='\t', index=False)

    # mask dist<2
    mask = cvd['dist'] < 2
    cvd.loc[mask, ['balanced.avg.smoothed',
                   'balanced.avg.smoothed.agg']] = np.nan

    # merge duplicates & save
    cvd_merged = cvd.drop_duplicates(subset=['dist'])[
        ['dist_bp', 'balanced.avg.smoothed.agg']
    ]
    out_merged = os.path.join(
        OUTDIR,
        f'{NAME}_{RES}bp_downsampled-{DSCOUNT//1_000_000}M_cvd_smooth_merged.tsv'
    )
    cvd_merged.to_csv(out_merged, sep='\t', index=False)

    # derivative in log-log space & save
    der = np.gradient(
        np.log(cvd_merged['balanced.avg.smoothed.agg']),
        np.log(cvd_merged['dist_bp'])
    )
    der_df = pd.DataFrame({'derivative': der})
    out_deriv = os.path.join(
        OUTDIR,
        f'{NAME}_{RES}bp_downsampled-{DSCOUNT//1_000_000}M_cvd_smooth_merged_derivative.tsv'
    )
    der_df.to_csv(out_deriv, sep='\t', index=False)

    print('Done.')
    return cvd, cvd_merged, der_df

In [11]:
# names = [
#     'G1DMSO_pooled',
#     'G1dTAG_pooled',
#     'G1A485_pooled',
#     'GSE178982_AsyncUT_pooled',
#     'GSE178982_AsyncAID_pooled'
# ]

names = [
    'GSE178982_AsyncUT_pooled',
    'GSE178982_AsyncAID_pooled'
]

for name in names:
    print(f"Processing {name}…")
    process_cvd(
        MCOOLDIR    = '../data/mcool_pooled',
        COOLDIR     = '../data/cool_norm_pooled',
        OUTDIR      = '../data/cvd',
        NAME        = name,
        RES         = 1000,
        DSCOUNT     = 5_000_000,
        mm10_arms   = mm10_arms,
        num_cpus    = num_cpus,
        WORKDIR     = WORKDIR
    )

Processing GSE178982_AsyncUT_pooled…
prenorm: 1820340.4290344473
sampling fraction: 2.7467389726943114
sampling to ../data/cool_norm_pooled/GSE178982_AsyncUT_pooled_1000bp_downsampled-5M.cool


INFO:root:creating a Pool of 10 workers


ValueError: The number of contacts in a sample cannot exceed that in the original dataset.

In [15]:
COOLDIR     = '../data/mcool_pooled'
NAME        = 'G1DMSO_pooled_allRes'
RES         = 1000
pre_norm = cooler.Cooler(f'{COOLDIR}/{NAME}.mcool::/resolutions/1000')
pre_norm.info

{'bin-size': 1000,
 'bin-type': 'fixed',
 'creation-date': '2025-05-16T19:21:18.889780',
 'format': 'HDF5::Cooler',
 'format-url': 'https://github.com/4dn-dcic/hic2cool',
 'format-version': 3,
 'generated-by': 'hic2cool-0.6.0',
 'genome-assembly': '/athena/apostoloulab/scratch/collab/Genome/Mus_Musculus/UCSC/mm10/Sequence/WholeGenomeFasta/genome.sizes',
 'hicFileScalingFactor': 1.0,
 'nbins': 2725549,
 'nchroms': 22,
 'nnz': 902251031,
 'nviIndex': '0000000000000000',
 'nviLength': '0000000000000000',
 'software': 'Juicer Tools Version 1.22.01',
 'storage-mode': 'symmetric-upper'}

In [16]:
pre_norm.bins()[:]

,chrom,start,end,KR,SCALE,VC,VC_SQRT
0,chr1,0,1000,NaN,NaN,0.0,0.0
1,chr1,1000,2000,NaN,NaN,0.0,0.0
2,chr1,2000,3000,NaN,NaN,0.0,0.0
3,chr1,3000,4000,NaN,NaN,0.0,0.0
4,chr1,4000,5000,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...
2725544,chrY,91740000,91741000,NaN,NaN,0.0,0.0
2725545,chrY,91741000,91742000,NaN,NaN,0.0,0.0
2725546,chrY,91742000,91743000,NaN,NaN,0.0,0.0
2725547,chrY,91743000,91744000,NaN,NaN,0.0,0.0


In [13]:
COOLDIR     = '../data/cool_norm_pooled'
NAME        = 'G1DMSO_pooled'
RES         = 1000
pre_norm = cooler.Cooler(f'{COOLDIR}/{NAME}_{RES}bp_KR.cool')
pre_norm.info

{'bin-size': 1000,
 'bin-type': 'fixed',
 'creation-date': '2025-05-17T10:54:28.102751',
 'format': 'HDF5::Cooler',
 'format-url': 'https://github.com/mirnylab/cooler',
 'format-version': 3,
 'generated-by': 'HiCMatrix-17.2',
 'generated-by-cooler-lib': 'cooler-0.10.2',
 'genome-assembly': '/athena/apostoloulab/scratch/collab/Genome/Mus_Musculus/UCSC/mm10/Sequence/WholeGenomeFasta/genome.sizes',
 'metadata': {'format': 'HDF5::Cooler',
  'format-url': 'https://github.com/mirnylab/cooler',
  'generated-by': 'HiCMatrix-17.2',
  'generated-by-cooler-lib': 'cooler-0.10.2',
  'tool-url': 'https://github.com/deeptools/HiCMatrix',
  'genome-assembly': '/athena/apostoloulab/scratch/collab/Genome/Mus_Musculus/UCSC/mm10/Sequence/WholeGenomeFasta/genome.sizes'},
 'nbins': 2725549,
 'nchroms': 22,
 'nnz': 901956778,
 'storage-mode': 'symmetric-upper',
 'sum': 1387522117.0,
 'tool-url': 'https://github.com/deeptools/HiCMatrix'}

In [14]:
pre_norm.bins()[:]

,chrom,start,end,weight
0,chr1,0,1000,0.0
1,chr1,1000,2000,0.0
2,chr1,2000,3000,0.0
3,chr1,3000,4000,0.0
4,chr1,4000,5000,0.0
...,...,...,...,...
2725544,chrY,91740000,91741000,0.0
2725545,chrY,91741000,91742000,0.0
2725546,chrY,91742000,91743000,0.0
2725547,chrY,91743000,91744000,0.0


In [23]:
COOLDIR     = '../data/mcool_pooled'
NAME        = 'GSE178982_AsyncAID_pooled_allRes'
RES         = 1000
pre_norm = cooler.Cooler(f'{COOLDIR}/{NAME}.mcool::/resolutions/1000')
pre_norm.info

{'bin-size': 1000,
 'bin-type': 'fixed',
 'creation-date': '2020-08-15T10:21:57.149892',
 'format': 'HDF5::Cooler',
 'format-url': 'https://github.com/mirnylab/cooler',
 'format-version': 3,
 'generated-by': 'cooler-0.8.6.post0',
 'genome-assembly': 'unknown',
 'metadata': {},
 'nbins': 2725549,
 'nchroms': 22,
 'nnz': 830296202,
 'storage-mode': 'symmetric-upper',
 'sum': 2088999279}

In [22]:
pre_norm.bins()[:]

,chrom,start,end,weight
0,chr1,0,1000,NaN
1,chr1,1000,2000,NaN
2,chr1,2000,3000,NaN
3,chr1,3000,4000,NaN
4,chr1,4000,5000,NaN
...,...,...,...,...
2725544,chrY,91740000,91741000,NaN
2725545,chrY,91741000,91742000,NaN
2725546,chrY,91742000,91743000,NaN
2725547,chrY,91743000,91744000,NaN


In [19]:
COOLDIR     = '../data/cool_norm_pooled'
NAME        = 'GSE178982_AsyncUT_pooled'
RES         = 1000
pre_norm = cooler.Cooler(f'{COOLDIR}/{NAME}_{RES}bp_KR.cool')
pre_norm.info

{'bin-size': 1000,
 'bin-type': 'fixed',
 'creation-date': '2024-08-22T10:09:48.351651',
 'format': 'HDF5::Cooler',
 'format-url': 'https://github.com/mirnylab/cooler',
 'format-version': 3,
 'generated-by': 'HiCMatrix-17.2',
 'generated-by-cooler-lib': 'cooler-0.10.2',
 'genome-assembly': 'unknown',
 'metadata': {'format': 'HDF5::Cooler',
  'format-url': 'https://github.com/mirnylab/cooler',
  'generated-by': 'HiCMatrix-17.2',
  'generated-by-cooler-lib': 'cooler-0.10.2',
  'tool-url': 'https://github.com/deeptools/HiCMatrix'},
 'nbins': 2725549,
 'nchroms': 22,
 'nnz': 278597354,
 'storage-mode': 'symmetric-upper',
 'sum': 1820340.4290344473,
 'tool-url': 'https://github.com/deeptools/HiCMatrix'}

In [20]:
pre_norm.bins()[:]

,chrom,start,end,weight
0,chr1,0,1000,345.77633
1,chr1,1000,2000,345.77633
2,chr1,2000,3000,345.77633
3,chr1,3000,4000,345.77633
4,chr1,4000,5000,345.77633
...,...,...,...,...
2725544,chrY,91740000,91741000,345.77633
2725545,chrY,91741000,91742000,345.77633
2725546,chrY,91742000,91743000,345.77633
2725547,chrY,91743000,91744000,345.77633


In [ ]:



Current problem is that the sum is not properlly labled in publish dataset. Check the sequencing depth first, it is only 601 M, and also check why

my mcool doesn't have sum, 
but has sum in the res specific extracted & normalized